In [449]:
class node:
    def __init__(self, name, num, right = None, left = None):
        self.right = right
        self.left  = left
        self.name  = name
        self.num   = num
    
    def __str__(self):
        return str((self.name, self.num, self.left, self.right))


class HuffmanCoding:
    def __init__(self, path):
        self.path = path
        self.frequency = {}
        self.codes = {}
        self.temp_code = ''
        self.write = '0'
        self.i = 0
        self.read = ''
        self.final_string = ''


    def compress(self):
        self.to_list()
        self.get_frequency()
        #for i in self.frequency:
        #    print(i)
        self.create_tree()
        self.create_codes(self.frequency[0], '')
        self.write_file()

        self.decompress()

    def to_list(self):
        with open(self.path, 'r') as f:
             self.list = f.read()
        self.list = list(self.list)


    def get_frequency(self):
        self.frequency = dict.fromkeys(list(dict.fromkeys(self.list))) #Deletes Repetitions and creates dict
        for c in self.frequency.keys():
            self.frequency[c] = self.list.count(c)
        self.frequency = sorted(self.frequency.items(), key=lambda x: x[1], reverse = False)

        for i in range(len(self.frequency)): #Turn them all into nodes
            self.frequency[i] = node(self.frequency[i][0], self.frequency[i][1])


    def create_tree(self):
        self.i += 1
        while(len(self.frequency) != 1):
            inserted = False
            parent = node("P" + str(self.i), self.frequency[0].num + self.frequency[1].num, self.frequency[0], self.frequency[1])
            #print("Making Node From ", self.frequency[0].name, " and ", self.frequency[1].name)

            del self.frequency[0]
            del self.frequency[0]

            for i in range(len(self.frequency)):
                if(self.frequency[i].num >= parent.num): 
                    if(not inserted):
                        #print("Inserting at: ", i)
                        self.frequency.insert(i, parent)
                        inserted = True
            if(not inserted):
                self.frequency.append(parent)

    def create_codes(self, subtree, code):
        if(subtree.right == None and subtree.left == None): #Found a letter
            #print("Code for ", subtree.name, ": ", code)
            self.codes[subtree.name] = code
            return
        else:
            self.create_codes(subtree.right, code + '0')
            self.create_codes(subtree.left,  code + '1')
            return

    def write_file(self):
        for c in self.list:
            self.write += self.codes[c]
       
        #Padding End:
        #self.write += (8 - len(self.write) % 8) * '0'

        #print("Writing: ", self.write)

        self.b = bytearray()
        for i in range(0, len(self.write), 8):
            byte = self.write[i:i+8]
            self.b.append(int(byte, 2))
        #print("Writing: ", self.b)
        with open(self.path.split('.')[0] + ".bin", "wb") as f:
            f.write(bytes(self.b))


    def decompress(self):
        #Extracts the binary and puts it into self.string
        with open(self.path.split('.')[0] + ".bin", "rb") as f:
            self.read = ''
            byte = f.read(1)
            while(byte):
                b = bin(ord(byte)).split('b')[1]
                #print(b)
                if(len(b) % 8 == 0):
                    self.read += b
                else:
                    #print("0" * (8 - len(b) % 8) + b)
                    self.read += "0" * (8 - len(b) % 8) + b
                byte = f.read(1)
            
        self.read = self.read[:-7]
        #print("Reading: ", self.read)
        self.read = self.read[1:]
        #print("Reading: ", self.read)

        self.decodes = {value : key for (key, value) in self.codes.items()}
        #print(self.decodes)
        word = ''
        for bit in self.read:
            word += bit
            if(word in self.decodes.keys()):
                self.final_string += self.decodes[word]
                word = ''
        print("Decoded:", self.final_string)

        #Write to Text
        with open(self.path.split('.')[0] + "_decoded.txt", "w") as f:
            f.write(self.final_string)
        #Needs padding
        #Encoding the number of bits you padded at the end as an 8 bit number at the start.


#Some Ideas: 
#What if you convert the image to like hexadecimal then run it through this?

In [453]:
h = HuffmanCoding('file4.txt')
h.compress()
#print(h.write)
#print(h.read)
#print(h.codes)
#print(h.read)

Decoded: Hell
{'l': '0', 'o': '10', 'H': '110', 'e': '111'}
